# Actividad 20 Datos con Pandas II

### Utilizando los conceptos aprendidos en el módulo 20 - Datos con Pandas II, resolver el siguiente ejercicio.

    Desarrollar una aplicación que realice la siguientes tareas:
    1. Conectarse a la url que contiene el archivo CSV de medallas
    olímpicas.
    2. Descargar los datos y obtener un sub dataset que contenga a
    todas las medallas de oro (Gold) Estados Unidos (USA) a partir del
    año 1950.
    3. Crear una base de datos “olympics” en SQLite y la tabla “medals”.
    4. Guardar en la base de datos. los datos generados en el sub
    dataset.
    5. Consultar la base de datos y validar que los datos se hayan
    cargado correctamente.

In [1]:
# Importar librerias necesarias
import sqlite3
import pandas as pd
import requests

In [3]:
# dirección web del archivo .csv
URL = 'http://winterolympicsmedals.com/medals.csv'

In [4]:
# Utilizando la libreria requests
# Descargar csv
# Descargamos el archivo
medallas = requests.get(URL)

# Guardamos el csv
with open('medals.cvs', 'w', encoding='utf-8') as f:
    f.write(medallas.text)

# Lo cargamos en el Dataframe
df_medallas = pd.read_csv('medals.cvs')
df_medallas

,Year,City,Sport,Discipline,NOC,Event,Event gender,Medal
0,1924,Chamonix,Skating,Figure skating,AUT,individual,M,Silver
1,1924,Chamonix,Skating,Figure skating,AUT,individual,W,Gold
2,1924,Chamonix,Skating,Figure skating,AUT,pairs,X,Gold
3,1924,Chamonix,Bobsleigh,Bobsleigh,BEL,four-man,M,Bronze
4,1924,Chamonix,Ice Hockey,Ice Hockey,CAN,ice hockey,M,Gold
...,...,...,...,...,...,...,...,...
2306,2006,Turin,Skiing,Snowboard,USA,Half-pipe,M,Silver
2307,2006,Turin,Skiing,Snowboard,USA,Half-pipe,W,Gold
2308,2006,Turin,Skiing,Snowboard,USA,Half-pipe,W,Silver
2309,2006,Turin,Skiing,Snowboard,USA,Snowboard Cross,M,Gold


In [5]:
# Tambien se puede con descargar el .csv con pandas, es más limitado pero funciona muy bien
try:
    df_medallas_pandas = pd.read_csv(URL)
except FileExistsError as ex:
    raise (ex)
df_medallas_pandas

,Year,City,Sport,Discipline,NOC,Event,Event gender,Medal
0,1924,Chamonix,Skating,Figure skating,AUT,individual,M,Silver
1,1924,Chamonix,Skating,Figure skating,AUT,individual,W,Gold
2,1924,Chamonix,Skating,Figure skating,AUT,pairs,X,Gold
3,1924,Chamonix,Bobsleigh,Bobsleigh,BEL,four-man,M,Bronze
4,1924,Chamonix,Ice Hockey,Ice Hockey,CAN,ice hockey,M,Gold
...,...,...,...,...,...,...,...,...
2306,2006,Turin,Skiing,Snowboard,USA,Half-pipe,M,Silver
2307,2006,Turin,Skiing,Snowboard,USA,Half-pipe,W,Gold
2308,2006,Turin,Skiing,Snowboard,USA,Half-pipe,W,Silver
2309,2006,Turin,Skiing,Snowboard,USA,Snowboard Cross,M,Gold


In [6]:
# usaremos df_medallas para continuar con las implementaciones
df_medallas.dtypes

Year             int64
City            object
Sport           object
Discipline      object
NOC             object
Event           object
Event gender    object
Medal           object
dtype: object

In [7]:
# Armando el sub dataset
df_subset_medallas = df_medallas[(df_medallas['NOC'] == 'USA') & (df_medallas['Medal'] == 'Gold') & (df_medallas['Year'] >= 1950)]
df_subset_medallas

,Year,City,Sport,Discipline,NOC,Event,Event gender,Medal
311,1952,Oslo,Skating,Figure skating,USA,individual,M,Gold
314,1952,Oslo,Skating,Speed skating,USA,500m,M,Gold
316,1952,Oslo,Skiing,Alpine Skiing,USA,giant slalom,W,Gold
317,1952,Oslo,Skiing,Alpine Skiing,USA,slalom,W,Gold
386,1956,Cortina d'Ampezzo,Skating,Figure skating,USA,individual,M,Gold
...,...,...,...,...,...,...,...,...
2301,2006,Turin,Skiing,Alpine Skiing,USA,Alpine combined,M,Gold
2302,2006,Turin,Skiing,Alpine Skiing,USA,giant slalom,W,Gold
2305,2006,Turin,Skiing,Snowboard,USA,Half-pipe,M,Gold
2307,2006,Turin,Skiing,Snowboard,USA,Half-pipe,W,Gold


In [10]:
# Armando base de datos 
# Creo conexión y el cursor
conn = sqlite3.connect('olympics.db')
cursor = conn.cursor()

In [11]:
# Tabla para guardar el df
query = """
CREATE TABLE medals(
    Year INTEGER,
    City TEXT,
    Sport TEXT,
    Discipline TEXT,
    NOC TEXT,
    Event Text,
    'Event gender' TEXT,
    Medal Text
)
"""
cursor.execute(query)
conn.commit()

In [12]:
# Guardando info con pandas y una conexion de sqlite3
# tambien se puede guardar por medio de sqlachemy pasandole en con el engine de sqlachemy
df_subset_medallas.to_sql(name='medals', con=conn, if_exists='append', index=False)

65

In [13]:
# Mostrando info
# Con pandas
query_pandas = "SELECT * FROM medals"
data_con_pandas = pd.read_sql(query_pandas, con=conn)
data_con_pandas

,Year,City,Sport,Discipline,NOC,Event,Event gender,Medal
0,1952,Oslo,Skating,Figure skating,USA,individual,M,Gold
1,1952,Oslo,Skating,Speed skating,USA,500m,M,Gold
2,1952,Oslo,Skiing,Alpine Skiing,USA,giant slalom,W,Gold
3,1952,Oslo,Skiing,Alpine Skiing,USA,slalom,W,Gold
4,1956,Cortina d'Ampezzo,Skating,Figure skating,USA,individual,M,Gold
...,...,...,...,...,...,...,...,...
60,2006,Turin,Skiing,Alpine Skiing,USA,Alpine combined,M,Gold
61,2006,Turin,Skiing,Alpine Skiing,USA,giant slalom,W,Gold
62,2006,Turin,Skiing,Snowboard,USA,Half-pipe,M,Gold
63,2006,Turin,Skiing,Snowboard,USA,Half-pipe,W,Gold


In [14]:
# Con Sqlite3
query_sqlite3 = "SELECT * FROM medals"
cursor.execute(query_sqlite3)
data_sqlite3 = cursor.fetchall()
# Cada letra representa la inicial de la columna en la tabla e imprime los primeros 20 resultados
for y, c, s, d, n, e, eg, m in data_sqlite3[:20]:
    print(f'{y} | {c} | {s} | {d} | {n} | {e} | {eg} | {m}')

1952 | Oslo | Skating | Figure skating | USA | individual | M | Gold
1952 | Oslo | Skating | Speed skating | USA | 500m | M | Gold
1952 | Oslo | Skiing | Alpine Skiing | USA | giant slalom | W | Gold
1952 | Oslo | Skiing | Alpine Skiing | USA | slalom | W | Gold
1956 | Cortina d'Ampezzo | Skating | Figure skating | USA | individual | M | Gold
1956 | Cortina d'Ampezzo | Skating | Figure skating | USA | individual | W | Gold
1960 | Squaw Valley | Ice Hockey | Ice Hockey | USA | ice hockey | M | Gold
1960 | Squaw Valley | Skating | Figure skating | USA | individual | M | Gold
1960 | Squaw Valley | Skating | Figure skating | USA | individual | W | Gold
1964 | Innsbruck | Skating | Speed skating | USA | 500m | M | Gold
1968 | Grenoble | Skating | Figure skating | USA | individual | W | Gold
1972 | Sapporo | Skating | Speed skating | USA | 1500m | W | Gold
1972 | Sapporo | Skating | Speed skating | USA | 500m | W | Gold
1972 | Sapporo | Skiing | Alpine Skiing | USA | slalom | W | Gold
1976 |

In [15]:
# Cerrando conexión
conn.close()

[Volver al Indice](../README.md)